In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import os
from dotenv import load_dotenv
import pymysql.cursors

In [ ]:
load_dotenv();

In [ ]:
plt.style.use('dark_background') 

def highlight_odd_rows(s):
    '''
    Changes style of a dataframe visualization
    Args:
       s : The row of the dataframe
    Returns:
    '''
    styles = []
    for i in range(len(s)):
        if i % 2 == 1:
            styles.append('background-color: indigo; border: 1px solid white;')
        else:
            styles.append('border: 1px solid white;')
    return styles

In [ ]:
class DataFrame_analysis:
    '''
    This class contains every method for regularization microservice function analysis
    '''
    #Class variables

    #The init method 
    def __init__(self):
        pass

    #Public
    def getDfSize(self, df):
        '''
        Returns the rows and columns of a dataframe
        Args:
           df (Dataframe) : Dataframe to analyze
        Returns:
          ... (dictionary): A dictionary with rows and columns as keys
        '''
        rows = len(df.axes[0])
        columns = len(df.axes[1])
        return {'rows': rows, 'columns': columns}

    #Public
    def determineDistinctValues(self, df, col):
        '''
        Determines the unrepeated values in a columns
        Args:
           df (DataFrame) : Dataframe which contains the column to search
           col (string) : Column's name to search
        Returns:
           df[col].unique() (array): An array (iterable object) with the unrepeated values of the column
        '''
        return df[col].unique()

    #Public
    def determineDistinctKeys(self, columnToList):
        '''
        Extracts the keys of a dictionary list and saves it in a dictionary with its count of appearences in the list
        Args:
           columnList (list) : List of dictionary to extract the keys
        Returns:
           distinctKeys (dictionary): A dictionary with the keys as a string and its appearences count in the list
        '''
        distinctKeysDict = {}
        distinctKeysList = []
        for dictionary in columnToList:
            dictionary = json.loads(dictionary)
            orderedKeys = sorted(list(dictionary.keys()))
            if orderedKeys in distinctKeysList:
                distinctKeysDict['/'.join(orderedKeys)] += 1
            else:
                distinctKeysList.append(orderedKeys)
                distinctKeysDict['/'.join(orderedKeys)] = 0
        return distinctKeysDict

    #Public
    def countAlertCodes(self, df, alertsColumn: str):
        '''
        Counts the alert codes in a column of a dataframe
        Args:
           df (DataFrame) : Dataframe which contains the column to search
           alertsColumn (string) : Column's name to search
        Returns:
           codeCount (dictionary): A dictionary with the distinct alert codes as keys and the appearence count in the column
        '''
        alertsWithCode = filter(lambda x: x != [],df['alertas'].values.tolist())
        codeCount = {'Code 1': 0 , 'Code 2': 0, 'Code 3': 0, 'Code 4': 0, 'Code 5': 0, 'Code 6': 0, 'Code 7': 0}

        for alert in alertsWithCode:
            alertList = json.loads(alert)
            if alertList != []:
                for subalert in alertList:
                    n = subalert['codigo']
                    codeCount[''.join(['Code ', str(n)])] += 1
    
        return codeCount

    #Public
    def calculateDeltaTime(self, df, columnA, columnB):
        '''
        Converts columnA and columnB in datetime type and makes the difference between both columns
        Args:
           df (DataFrame) : Dataframe which contains the columns to substraction
           columnA (string) : Minuend column
           columnB (string): Substrahend column
        Returns:
           responsePeriod (DataFrame): Returns a column of the seconds difference as a dataframe
        '''
        responseTimeType = pd.to_datetime(df[columnA], format='%Y-%m-%d %H:%M:%S')
        requestTimeType = pd.to_datetime(df[columnB], format='%Y-%m-%d %H:%M:%S')
        responsePeriod = (responseTimeType - requestTimeType).to_frame()
        responsePeriod['secondsDifference'] = pd.to_timedelta(responsePeriod[0]).dt.total_seconds()
        responsePeriod.drop([0], axis='columns', inplace=True)

        return responsePeriod

    #Public
    def proveErrorAlert(self, df):
        '''
        Assures every error in CarfaxUsaData is associated with an alert
        Args:
           df (DataFrame) : Dataframe which contains the columns to substraction
        Returns:
           ... (string): Returns a status message
        '''
        carfaxUsaData = df['carfaxUsaData'].values.tolist()
        conError = 0
        for index1 in range(len(carfaxUsaData)):
            carfaxDict = json.loads(carfaxUsaData[index1])
            if 'error' in list(carfaxDict.keys()):
                conError += 1
                if df.iloc[index1]['alertas'] == []:
                    return True
        return False

    #Private
    def getRepeatedValuesInAColumn(self, df, columnName):
        '''
        Identifies the repeated values of a column in a dataframe
        Args:
           df (DataFrame) : Dataframe which contains the columns to search
           columnName (string) : Column's name to search
        Returns:
           duplicates['VIN'] (pandas Series): Returns a column as a pandas series of the duplicated values of the columnName
        '''
        duplicates = df[df.duplicated(subset=[columnName], keep=False)]
        return duplicates['VIN']

    #Private
    def searchValuesInADataframe(self, df, values:list, columnA: str, columnB: str)-> dict:
        '''
        Optimized version to search values in a dataframe columnA and retrieve the corresponding values of columnB.
        Args:
           values (list): The list of values to be searched.
           columnA (str): The name of the column to search.
           columnB (str): The name of the column to retrieve values from.
           df (DataFrame): The dataframe to search in.
        Returns:
          grouped: A dictionary with keys as the values from columnA and values as the list of corresponding entries from columnB.
        '''
        
        filtered_df = df[df[columnA].isin(values)]
    
        grouped = filtered_df.groupby(columnA)[columnB].apply(list).to_dict()
    
        return grouped

    #Public
    def getIncorrectRepeatedVinInformation(self, df)->dict:
        '''
        Searches the values of a list in a dataframe columnA and retrieves the information of columnB
        Args:
           df (DataFrame): The dataframe where where will be searched
        Returns:
          ... (dict): A dictionary which contains a status message, a list of good and bad VINs withits appeareance count, and the total count of good and bad VINS
        '''
        repeatedRows = len(self.getRepeatedValuesInAColumn(df, 'VIN').values.tolist())
        vinValues = self.getRepeatedValuesInAColumn(df, 'VIN').unique().tolist()
        repeatedVins = len(vinValues)
        repeatedValues = self.searchValuesInADataframe(df, vinValues, 'VIN', 'alertas')
        goodVins = 0
        badVins = 0
        goodVinsList = []
        badVinsList = []
        for repeatedVin in list(repeatedValues.keys()):
            differentValuesByVin = list(set(repeatedValues[repeatedVin]))
            for differentValue in differentValuesByVin:
                if differentValue == '[]':
                    goodVins += 1
                    if len(differentValuesByVin)>1:
                        return {'message': ''.join(['Alerta con vin: ',repeatedVin, ' , tiene y no tiene alertas.']),
                               'goodVinsList': [],
                               'badVinsList': [],
                              'goodVins':0,
                               'badVins': 0}
                    else:
                        goodVinsList.append([repeatedVin, len(repeatedValues[repeatedVin])])
                else:
                    badVins += 1
                    badVinsList.append([repeatedVin, len(repeatedValues[repeatedVin])])
            

        return {'message': '',
               'goodVinsList': goodVinsList,
               'badVinsList': badVinsList,
               'goodVins':goodVins,
               'badVins': badVins}

    #Public
    def diagnoseVins(self, df):
        '''
        Converts the lists of  getIncorrectRepeatedVinInformation() function in dataframes
        Args:
           df (DataFrame): The dataframe where where will be searched
        Returns:
          ... (dict): A dictionary which contains the good and bad VINs dataframes and the good and bad repeated Records of the VINs
        '''
        goodVinsFrame = pd.DataFrame(self.getIncorrectRepeatedVinInformation(df)['goodVinsList'], columns = ['VIN', 'repeatedTimes'])
        goodRepeatedRecords = goodVinsFrame.sum()['repeatedTimes']
        badVinsFrame = pd.DataFrame(self.getIncorrectRepeatedVinInformation(df)['badVinsList'], columns = ['VIN', 'repeatedTimes'])
        badRepeatedRecords = badVinsFrame.sum()['repeatedTimes']

        return {'goodVinsFrame': goodVinsFrame, 'badVinsFrame': badVinsFrame, 'goodRepeatedRecords': goodRepeatedRecords, 'badRepeatedRecords': badRepeatedRecords}

    def verifyInfoRepeatedVins(self, df):
        '''
        Verifies every repeated VIN has the same information in every record (due to the short period time reviewed)
        Args:
           df (DataFrame): The dataframe where where will be searched
        Returns:
          ... (dict): A dictionary which contains the different responses as a dict for every repeated VIN in uniqueResponses and VINs with more than one response in differentResponse key
        '''
        uniqueResponses = {}
        differentResponses = {}
        vinValues = self.getRepeatedValuesInAColumn(df, 'VIN').unique().tolist()
        repeatedValues = self.searchValuesInADataframe(df, vinValues, 'VIN', 'responseBody')
        repeatedVins = repeatedValues.keys()
        for vin in repeatedVins:
            uniqueResponses[vin] = []
            listJson = []
            for responseString in repeatedValues[vin]:
                responseJson = json.loads(responseString)
                listJson.append(responseJson)
            repeatedValues[vin] = listJson
        for vin in repeatedVins:
            for responseJson in repeatedValues[vin]:
                keys = list(responseJson.keys())
                if 'fabricante' in keys and 'paisOrigen' in keys:
                    info = {'anioModelo': responseJson['anioModelo'], 'fabricante': responseJson['fabricante'], 'marca': responseJson['marca'], 'modelo': responseJson['modelo'], 'paisOrigen': responseJson['paisOrigen'], 'robo': responseJson['robo'], 'roboFecha': responseJson['roboFecha'], 'codes': []}
                else:
                     info = {'anioModelo': responseJson['anioModelo'], 'marca': responseJson['marca'], 'modelo': responseJson['modelo'], 'robo': responseJson['robo'], 'roboFecha': responseJson['roboFecha'], 'codes': []}
                if  responseJson['mensajes'] != []:
                    for message in responseJson['mensajes']:
                        info['codes'].append(message['codigo'])
                        info['codes'] = list(set(info['codes']))
                    if len(info['codes'])>1:
                        info['codes'] = info['codes'].sort()
                if uniqueResponses[vin] == []:
                     uniqueResponses[vin].append(info)
                else:
                    if info not in uniqueResponses[vin]:
                        uniqueResponses[vin].append(info)
                        differentResponses[vin] = info
        return {'uniqueResponses': uniqueResponses, 'differentResponses': differentResponses}

In [ ]:
#### Here, we will connect to the database, make the query and convert table into dataframe for analysis
# SSL configuration
ssl_config = {
    #"ssl_ca": os.getenv('CA-CERTIFICATE'),
    #"ssl_cert": os.getenv('CLIENT-CERTIFICATE'),
    #"ssl_key": os.getenv('CLIENT-KEY')
    "xd": True
}

db_config = {
    "user": os.getenv('USER'),
    "password": os.getenv('PASSWORD'),
    "host": os.getenv('HOST'),  
    "port": int(os.getenv('PORT')),
    "ssl":ssl_config
} 

print(db_config)

try:    
    conn = pymysql.connect(**db_config)    
    print(conn)
    cursor = conn.cursor()
    
    # Execute a query
    cursor.execute('SHOW DATABASES;')
    result = cursor.fetchall()
    print(f"Connected to database: {result}")
    
    # Close the cursor and connection
    cursor.close()
    conn.close()

except pymysql.Error as err:
    print(err)

In [ ]:
regularizationFrame = pd.read_csv("regularizacion100000.csv", encoding = 'utf-8').sort_values(by = 'responseTime', ascending = False)

In [ ]:
filteredAnalysis = DataFrame_analysis()
n_minutes = 15
maxTimeAllowed = 6
totalRows = filteredAnalysis.getDfSize(regularizationFrame)['rows']
partialRows = 1
recordsAnalysis = []
codesAnalysis = []
vinsAnalysis = []
regularizationFrame['responseTime'] = pd.to_datetime(regularizationFrame['responseTime'], format='%Y-%m-%d %H:%M:%S')
dataframeIndex=0


while partialRows < totalRows:
    #DATA SEPARATION BY TIME LAPSE n_minutes
    first_timestamp = regularizationFrame['responseTime'].iloc[partialRows-1]
    print('first', first_timestamp)
    time_threshold = first_timestamp - pd.Timedelta(minutes=n_minutes)
    filtered_df = regularizationFrame[(regularizationFrame['responseTime'] > time_threshold) & (regularizationFrame['responseTime'] <= first_timestamp)].fillna('xD')
    print('first', filtered_df['responseTime'].values[0:1], filtered_df['idRespuesta'].values[0:1])
    print('last',filtered_df['responseTime'].values[-1:], filtered_df['idRespuesta'].values[-1:])
    timeLapse = '- '.join([str(first_timestamp),str(time_threshold)])
    
    ##SHAPE DATA ANALYSIS
    if list(filteredAnalysis.determineDistinctKeys(filtered_df['requestParameters']).keys()) !=['']:
        print('Existen peticiones con RequestParameters inconvencionales')
    if list(filteredAnalysis.determineDistinctValues(filtered_df, 'apiKey'))!=['6cNqbbTqBnZEchPZ2YYn']:
        print('Existen peticiones con distinta ApiKey')  
    if list(filteredAnalysis.determineDistinctValues(filtered_df, 'userId'))!=['xD']:
        print('Existen peticiones con distinto userId')  
    if list(filteredAnalysis.determineDistinctValues(filtered_df, 'responseCode'))!=[200] and filteredAnalysis.determineDistinctValues(filtered_df, 'responseCodeStatus').values.list()!=['ok']:
        print('Existen peticiones con distinto responseCode')
    if list(filteredAnalysis.determineDistinctValues(filtered_df, 'labels'))!=['xD']:
        print('Existen peticiones con distinto labels')
    if list(filteredAnalysis.determineDistinctValues(filtered_df, 'firewallUsa'))!=[0]:
        print('Existen peticiones con distinto firewallUsa')
    totalRecords = filteredAnalysis.getDfSize(filtered_df)['rows']
    
    ##CONTENT DATA ANALYSIS
    
    #Alerts Analysis
    codeCount = filteredAnalysis.countAlertCodes(filtered_df, 'alertas')
    codesAnalysis.append([timeLapse])
    for code in codeCount.keys():
        codesAnalysis[dataframeIndex].append(codeCount[code])

    #Response Time Analysis
    responseTime = filteredAnalysis.calculateDeltaTime(filtered_df, 'responseTime', 'requestTime')
    #plt.hist(responseTime['secondsDifference'], bins = 20, edgecolor = 'black')
    #plt.xlabel('Time in Seconds')
    #plt.ylabel('Frequency')
    #plt.title(f'Histogram of Delay in Seconds {''.join([timeLapse.split(' ')[1],timeLapse.split(' ')[3]])}')
    #plt.show()

    maxTimeResponse = responseTime.max().values.tolist()[0]
    if maxTimeResponse > maxTimeAllowed:
        print('Existen peticiones que están tardando en responder más de lo usual: ', str(maxTimeResponse))

    #Congruence CarfaxUsaData-Alerts
    if filteredAnalysis.proveErrorAlert(filtered_df):
        print('Existen registros con error y sin alerta')

    #Good-Bad VINs Relation
    vinsRelations = filteredAnalysis.getIncorrectRepeatedVinInformation(filtered_df)
    if vinsRelations['message']!='':
        print( vinsRelations['message'])
    else:
        goodVins=vinsRelations['goodVins']
        badVins=vinsRelations['badVins']
        vinsAnalysis.append([timeLapse, badVins+goodVins, goodVins, badVins])
        
    #Good-Bad Records Relations
    framesSources = filteredAnalysis.diagnoseVins(filtered_df)
    goodRecords = filteredAnalysis.diagnoseVins(filtered_df)['goodRepeatedRecords']
    badRecords = filteredAnalysis.diagnoseVins(filtered_df)['badRepeatedRecords']
    radio = badRecords/totalRecords
    recordsAnalysis.append([timeLapse, totalRecords, goodRecords, badRecords, radio])

    #Review congruent information Repeated Vins
    if filteredAnalysis.verifyInfoRepeatedVins(filtered_df)['differentResponses'] != {}:
         print('Existen respuestas distintas para un mismo VIN: ', verifyInfoRepeatedVins(df)['differentResponses'])
     
    partialRows += totalRecords  
    print(partialRows)
    dataframeIndex+=1

%time

In [ ]:
vinsFrame = pd.DataFrame(vinsAnalysis, columns=['timeLapse', 'total', 'right', 'wrong'])
recordsFrame = pd.DataFrame(recordsAnalysis, columns = ['timeLapse', 'total', 'right', 'wrong', 'radio'])
codesFrame = pd.DataFrame(codesAnalysis, columns = ['timeLapse', 'code1', 'code2', 'code3', 'code4', 'code5', 'code6', 'code7'])

In [ ]:
plt.plot(vinsFrame['timeLapse'], vinsFrame['total'], marker='o', linestyle='-', color='w', label = 'Total Repeated Vins vs Time Lapse')
plt.plot(vinsFrame['timeLapse'], vinsFrame['right'], marker='o', linestyle='-', color='c', label = 'Right Repeated Vins vs Time Lapse')
plt.plot(vinsFrame['timeLapse'], vinsFrame['wrong'], marker='o', linestyle='-', color='y', label = 'Wrong Repeated Vins vs Time Lapse')
plt.xlabel('Time Lapse')
plt.ylabel('Repeated VINs')
plt.title('Graph of Repeated VINs')
plt.legend()
plt.show()

In [ ]:
plt.plot(recordsFrame['timeLapse'], recordsFrame['total'], marker='o', linestyle='-', color='w', label = 'Total Repeated Records vs Time Lapse')
plt.plot(recordsFrame['timeLapse'], recordsFrame['right'], marker='o', linestyle='-', color='c', label = 'Right Repeated Records vs Time Lapse')
plt.plot(recordsFrame['timeLapse'], recordsFrame['wrong'], marker='o', linestyle='-', color='y', label = 'Wrong Repeated Records vs Time Lapse')
plt.xlabel('Time Lapse')
plt.ylabel('Repeated Records')
plt.title('Graph of Repeated Records')
plt.legend()
plt.show()

In [ ]:
plt.plot(recordsFrame['timeLapse'], recordsFrame['radio'], marker='o', linestyle='-', color='w', label = 'Alert Radio')
plt.xlabel('Time Lapse')
plt.ylabel('Proportion')
plt.title('Graph of Bad Records Proportion')
plt.legend()
plt.show()

In [ ]:
plt.plot(codesFrame['timeLapse'], codesFrame['code7'], marker='o', linestyle='-', color='w', label = 'Code 7 Appearences vs Time Lapse')
plt.plot(codesFrame['timeLapse'], codesFrame['code6'], marker='o', linestyle='-', color='c', label = 'Code 6 Appearences vs Time Lapse')
plt.plot(codesFrame['timeLapse'], codesFrame['code4'], marker='o', linestyle='-', color='y', label = 'Code 4 Appearences vs Time Lapse')
plt.plot(codesFrame['timeLapse'], codesFrame['code5'], marker='o', linestyle='-', color='r', label = 'Code 5 Appearences vs Time Lapse')
plt.xlabel('Time Lapse')
plt.ylabel('Repeated Records')
plt.title('Graph of Alert Code Appearences')
plt.legend()
plt.show()

In [ ]:
codesFrame.sum()

In [ ]:
recordsFrame.sum()

In [ ]:
recordsFrame[['timeLapse', 'radio']].describe()

In [ ]:
vinsFrame.sum()

## 